In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
# import pandas as pd
import re

In [3]:
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()


In [4]:
my_input_data = spark.read.option('header', True).csv("./disease_symptoms.csv")

for column in my_input_data.columns:
    new_col = re.sub(r'[^A-Za-z0-9 ]+', '', str(column))
    my_input_data = my_input_data.withColumnRenamed(column, new_col)

### My Custom Value Processor

In [5]:
def upperCase(str):
    return str.upper()


upperCaseUDF = udf(lambda x:upperCase(x),StringType())

In [7]:
# for column in my_input_data.columns:
#     my_input_data = my_input_data.withColumn(column, my_custom_function(col(column)))
my_input_data.printSchema()

          


root
 |-- Code: string (nullable = true)
 |-- name: string (nullable = true)
 |-- symptoms: string (nullable = true)
 |-- Desc: string (nullable = true)
 |-- commonTestsAndProceduresDesc: string (nullable = true)
 |-- commonTestsAndProcedures: string (nullable = true)
 |-- commonMedicationsDesc: string (nullable = true)
 |-- commonMedications: string (nullable = true)
 |-- whoIsAtRiskDesc: string (nullable = true)
 |-- symptomsDesc: string (nullable = true)

